# Asset-based lending: "a primer in python"<br>
>## <font color = green>Asset_based lending (ABL) is an efficient and flexible technique to finance working capital especially for leveraged businesses that need to manage cash.  It works quite well for noninvestment grade manufacturers and distributors that want to finance the growth of inventories and accounts.  In this notebook I've written a simple ABL platform in python to illustrate ABL's capabilities.
</font>

>__[ABL from Wikipedia](https://en.wikipedia.org/wiki/Asset-based_lending)__




>## <font color = 'blue'>Managing cash drives ABL. For simplicity sake we are only using accounts receivable as collateral..Here are a few critical points to understand about our model:
>*  The model track three primary cash flows:
    * <b>Presentments.</b>The line of credit funds checks presented for payment, outgoing wires and ACH transfers out.  In our model the method loan.draw(presentments) is used to fund cash outflows and results in a increase on the line of credit.
    * <b>Cash collections</b>.  The function, sales_collections(collateral_object, loan_object, collections) takes cash collections and uses it to reduce the loan balance and the accounts receivable.
    * <b>New Sales</b>.  The method Accounts_receivable.new_sales(sales) increase the accounts receivable
*  The model also tracks two stock accounts:
    * <b>Loan account.</b>  The loan consisting of the commited facility and the loan balance.  
    * <b>Collateral account.</b>  The total amount of accounts receivable.
* Lastly and most importantly the model provides a tool to company management and lending risk officers to monitor and track liquidity
    * <b>Availability.</b>  Calculates the maximun loan value of the collateral.  The class method Accounts_receivable.loan_availability calculates the loan value of the collateral
    * <b>Excess Availability.</b>  The function excess_availability(collateral_object, loan_object) nets the value of the collateral against the outstanding loan with the difference being excess availability.  Almost every method and function in our model impacts this number</>


### Step one.  import our dependencies

In [63]:
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt


In [64]:
from loan_sys_objects import Customer, Loan, Accounts_receivable, Interest
from loan_sys_functions import excess_availability, sales_collections, fund_presentments, daily_build
from loan_sys_functions import loan_hist, date_hist_str, i_rate_hist, accrued_interest_hist, liquidity_hist

### Step two.  Create an instance of the following:
* Customer object
* Loan object
* Accounts_receivable object
* Interest object


In [65]:
ABC_mfg = Customer('ABC_mfg', 'abc@gmail.com', 'Atlanta')
ABC_loan = Loan(5000000, 0)
ABC_collateral = Accounts_receivable(6000000, 600000, .85)
ABC_interest = Interest(.03, .0325)

#### Verify the customer object:

In [66]:
ABC_mfg

Customer name: ABC_mfg
Customer email: abc@gmail.com
Customer Location: Atlanta 

#### Verify the loan object:

In [67]:
ABC_loan

Commitment:$5,000,000
Loan: $0

#### Verify the collateral object:

In [68]:
ABC_collateral

Total Aging:$6,000,000
Past Dues: $600,000
Advance Rate: 85.00%

#### Verify the interest object

In [69]:
ABC_interest

Base Rate:3.00%
Margin: 3.25%

In [70]:
f'The interest rate is the sum of the base rate plus the margin: {ABC_interest.interest_rate():.2%}'

'The interest rate is the sum of the base rate plus the margin: 6.25%'

### <font color = green > To summarize:
* ABC mfg has a commited credit facility secured by the company's accounts receivable.  The initial loan balance is $0.
* ABC mfg has pledged 6,000,000 of accounts to the lender but the lender but determined that past due accounts are 600,000 and will lend 85 percent of the eligible accounts.
* The interest rate consists of two parts, a base rate and a margin.  The base rate is typically tied to an index such as 90 libor, US prime rate or some similar market rate.  The number is variable, that is it can fluctuate with the market.  The lenders margin is 3.25 percent above the index.  This percentage is contractually fixed.</font>



> #### An important concept is the term "availability".  For many borrowers excess availability is synonomous with liquidity

In [71]:
collateral_availability = ABC_collateral.loan_availability()
f'Collateral Availability totals ${collateral_availability:,.2f}.  This equates to gross acounts minus past dues and subsequently reduced by the advance rate of 85%'

'Collateral Availability totals $4,590,000.00.  This equates to gross acounts minus past dues and subsequently reduced by the advance rate of 85%'

In [72]:
excess_avail = excess_availability(ABC_collateral, ABC_loan)
f'Excess Availability totals ${excess_avail:,.2f}.  Excess availability equates to collateral availability less the loan balance'

'Excess Availability totals $4,590,000.00.  Excess availability equates to collateral availability less the loan balance'

### So far, excess Availability is the same as total Availability that's true because the borrower has a zero loan balance.  Lets borrow some money and see what happens

#### We will call the loan_draw method on the loan object to make a $50,000 loan advance

In [73]:
f"The borrower draws down $50,000 and the new loan balance is ${ABC_loan.loan_draw(50000):,.2f}"

'The borrower draws down $50,000 and the new loan balance is $50,000.00'

In [74]:
f" After the draw the excess availability has dropped $50,000 to ${excess_availability(ABC_collateral, ABC_loan):,.2f}"

' After the draw the excess availability has dropped $50,000 to $4,540,000.00'

In [75]:
ABC_loan

Commitment:$5,000,000
Loan: $50,000

In [76]:
ABC_collateral

Total Aging:$6,000,000
Past Dues: $600,000
Advance Rate: 85.00%

#### Next the borrower collects $10,000 of accounts.  We apply the collections to reduce the loan balance and to reduce the collateral balance

In [77]:
sales_collections(ABC_collateral, ABC_loan, 10000)

In [78]:
f"The new loan amount is ${ABC_loan.loan:,.2f}" 

'The new loan amount is $40,000.00'

In [79]:
f"The new excess availability is ${excess_availability(ABC_collateral, ABC_loan):,.2f}"

'The new excess availability is $4,541,500.00'

#### Let's recap:

In [80]:
ABC_loan

Commitment:$5,000,000
Loan: $40,000

In [81]:
ABC_collateral

Total Aging:$5,990,000
Past Dues: $600,000
Advance Rate: 85.00%

In [82]:
excess_avail = excess_availability(ABC_collateral, ABC_loan)
f'Excess Availability totals ${excess_avail:,.2f}.'

'Excess Availability totals $4,541,500.00.'

> ### <font color = 'green'> Once again to summarize:
* The borrower opened with no loan and 4,590,000 in excess undrawn availability on the line of credit.
* The borrower then took a draw for 50,000.  The loan went up and excess availability went down, both by 50,000
* Next the borrower collected 10,000 of accounts receivable. The loan decreases by $10,000 and the collateral decreases by $10,000 but the excess availability, the liquidity of the business increases by 1,500.

In [83]:
# date_hist = []
# loan_hist = []
# i_rate_hist = []
# accrued_interest_hist = []

In [97]:
fund_presentments(ABC_collateral, ABC_loan, 10000000)

Your advance of $10,000,000.00 exceeds your excess_availability of $3,540,500.00:  ADVANCE REJECTED!


In [85]:
ABC_loan.loan

41000

In [86]:
loan_hist

[0,
 1000,
 1000,
 5500,
 6500,
 7500,
 8250,
 9015.0,
 9795.300000000001,
 10591.206,
 11403.030120000001,
 12231.0907224,
 13075.712536848,
 13937.22678758496,
 14815.971323336658,
 15712.290749803393,
 16626.536564799462,
 17559.067296095454,
 18510.248642017366,
 19480.453614857714,
 20470.06268715487,
 21479.463940897967,
 22509.053219715926,
 23559.234284110247,
 24630.418969792452,
 25723.0273491883]

In [87]:
date_hist_str

[datetime.date(2018, 12, 31),
 datetime.date(2019, 1, 4),
 datetime.date(2019, 1, 5),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6),
 datetime.date(2019, 1, 6)]

In [88]:
i_rate_hist

[0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1]

In [89]:
accrued_interest_hist

[0,
 0.2777777777777778,
 0.2777777777777778,
 1.5277777777777777,
 1.8055555555555556,
 2.0833333333333335,
 2.2916666666666665,
 2.504166666666667,
 2.720916666666667,
 2.9420016666666666,
 3.167508366666667,
 3.397525200666667,
 3.6321423713466667,
 3.8714518854402664,
 4.115547589815739,
 4.364525208278721,
 4.6184823791109615,
 4.877518693359848,
 5.1417357338937135,
 5.411237115238254,
 5.686128524209686,
 5.966517761360547,
 6.252514783254425,
 6.54423174558618,
 6.84178304716457,
 7.145285374774529]

In [90]:
liquidity_hist

[0,
 32750.0,
 32375.0,
 27500.0,
 25750.0,
 24000.0,
 22687.5,
 21348.75,
 19983.225,
 18590.389499999997,
 17169.697289999996,
 15720.591235799995,
 14242.503060515997,
 12734.853121726317,
 11197.050184160844,
 9628.491187844058,
 8028.561011600941,
 6396.632231832955,
 4732.064876469609,
 3034.2061739989986,
 1302.3902974789744,
 -464.0618965714457,
 -2265.843134502873,
 -4103.659997192932,
 -5978.233197136789,
 -7890.297861079529]

In [91]:
len(date_hist_str), len(loan_hist), len(i_rate_hist), len(accrued_interest_hist)
# date_hist_str = [day.strftime for day in date_hist]
# date_hist_str = [i.strftime('%b-%d') for i in date_hist]
date_hist_str = [i.strftime('%b-%d') for i in date_hist_str]

In [92]:
date_hist_str

['Dec-31',
 'Jan-04',
 'Jan-05',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06',
 'Jan-06']

In [93]:
# df = pd.DataFrame(
#     {'loan':loan_hist,
#      'Interest Rate': i_rate_hist,
#      'Interest Accrual': accrued_interest_hist,
#      'Liquidity': liquidity_hist
#     }, index = date_hist)

df = pd.DataFrame(
    {'loan':loan_hist,
     'Interest Rate': i_rate_hist,
     'Interest Accrual': accrued_interest_hist,
     'Liquidity': liquidity_hist,
     'Date': date_hist_str
    })

In [94]:
df

,Date,Interest Accrual,Interest Rate,Liquidity,loan
0,Dec-31,0.000000,0.0,0.000000,0.000000
1,Jan-04,0.277778,0.1,32750.000000,1000.000000
2,Jan-05,0.277778,0.1,32375.000000,1000.000000
3,Jan-06,1.527778,0.1,27500.000000,5500.000000
4,Jan-06,1.805556,0.1,25750.000000,6500.000000
5,Jan-06,2.083333,0.1,24000.000000,7500.000000
6,Jan-06,2.291667,0.1,22687.500000,8250.000000
7,Jan-06,2.504167,0.1,21348.750000,9015.000000
8,Jan-06,2.720917,0.1,19983.225000,9795.300000
9,Jan-06,2.942002,0.1,18590.389500,10591.206000


In [95]:
%matplotlib inline
# plt.scatter(loan_hist, date_hist)
df.plot(kind='scatter',x='Date',y='Liquidity',color='red')
plt.show()

ValueError: scatter requires x column to be numeric

In [ ]:
ABC_loan

In [ ]:
excess_availability(ABC_collateral, ABC_loan)